In [1]:
from langchain.vectorstores import FAISS
from FlagEmbedding import FlagAutoModel
import torch

embed_model = FlagAutoModel.from_finetuned('BAAI/bge-base-en-v1.5',
                                      query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                                      use_fp16=True)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


model = AutoModelForCausalLM.from_pretrained(
    "thesven/Mistral-7B-Instruct-v0.3-GPTQ"
)
tokenizer = AutoTokenizer.from_pretrained("thesven/Mistral-7B-Instruct-v0.3-GPTQ")


INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


`low_cpu_mem_usage` was None, now default to True since model is quantized.


INFO  skip BACKEND.MARLIN for No module named 'gptqmodel_marlin_kernels'       
INFO  skip BACKEND.EXLLAMA_V2 for No module named 'gptqmodel_exllamav2_kernels'
INFO  skip BACKEND.EXLLAMA_V1 for No module named 'gptqmodel_exllama_kernels'  
INFO  skip BACKEND.TRITON for Trying to use the triton backend, but it could not be imported. Please install triton by 'pip install gptqmodel[triton] --no-build-isolation'
INFO  skip BACKEND.IPEX for <class 'Exception'>                                
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at thesven/Mistral-7B-Instruct-v0.3-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.24.mlp.down_proj.bias', 'model.layers.19.self_attn.v_proj.bias', 'model.layers.4.self_attn.o_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.28.mlp.down_proj.bias', 'model.layers.12.mlp.up_proj.bias', 'model.layers.18.self_attn.v_proj.bias', 'model.layers.27.mlp.up_proj.bias', 'model.layers.30.mlp.gate_proj.bias', 'model.layers.18.self_attn.q_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.3.mlp.up_proj.bias', 'model.layers.4.mlp.down_proj.bias', 'model.layers.26.self_attn.o_proj.bias', 'model.layers.3.self_attn.q_proj.bias', 'model.layers.9.self_attn.v_proj.bias', 'model.layers.14.self_attn.k_proj.bias', 'model.layers.27.mlp.down_proj.bias', 'model.layers.28.self_attn.k_proj.bias', 'model.layers.22.self_a

INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Converting GPTQ v1 to v2  

In [5]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [4]:
query = "ACETAMINOPHEN uses"
embed_query = embed_model.encode([query])

c:\Users\Jason\Documents\GitHub\EZ-Rx-ID\.venv\Lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [6]:
vector_store = FAISS.load_local('my_vector_store', embeddings=embed_model, allow_dangerous_deserialization=True)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.


In [7]:
if "pregnant" in query.lower():
    category = 'specific population usage'

elif "uses" in query.lower():
    category = 'general'

else:
    category = 'general'

In [8]:
results = vector_store.similarity_search_by_vector(
    embed_query[0], 
    k=3, 
    filter={"category": category}  
)

results

[Document(id='aee4e363-399e-4746-b59e-3a22251d7d02', metadata={'type': 'summary', 'category': 'general'}, page_content='Summary of ACETAMINOPHEN | Uses: Uses For the temporary relief of minor aches and pains associated with • headache • toothache • minor arthritis pain • muscular aches • common cold • menstrual cramps For the reduction of fever. and  |  and  | Reactions: '),
 Document(id='3fc85fc6-8aed-42e2-8689-745f01a9c628', metadata={'type': 'summary', 'category': 'general'}, page_content='Summary of ACETAMINOPHEN | Uses: Uses temporarily: • reduces fever • relieves minor aches and pains due to: • the common cold • flu • headache • sore throat • toothache and  |  and  | Reactions: '),
 Document(id='1f6688c3-daf1-49f7-982a-91cee986d21c', metadata={'type': 'summary', 'category': 'general'}, page_content='Summary of ACETAMINOPHEN | Uses: Uses temporarily: • reduces fever • relieves minor aches and pains due to: • the common cold • flu • headache • sore throat • toothache and  |  and  |

In [9]:
prompt_template = """
You are an expert medical assistant explaining medicine to someone without any prior knowledge. 
Use the provided context to answer the user's question accurately.

Context: {context}

Question: {question}

Answer:
"""

In [10]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
mistral_llm = HuggingFacePipeline(
    pipeline=pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
)

streaming_callback = StreamingStdOutCallbackHandler()

Device set to use cpu
C:\Users\Jason\AppData\Local\Temp\ipykernel_8504\1602439417.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  mistral_llm = HuggingFacePipeline(


In [11]:
qa_chain = prompt | mistral_llm

In [13]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [14]:
response = qa_chain.invoke({'context' : results, 'question' : query},
                           config={'callbacks' : [streaming_callback]})

response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
W0312 13:09:12.550000 8504 Lib\site-packages\torch\_dynamo\convert_frame.py:1233] WON'T CONVERT dequantize_weight c:\Users\Jason\Documents\GitHub\EZ-Rx-ID\.venv\Lib\site-packages\gptqmodel\nn_modules\qlinear\__init__.py line 437 
W0312 13:09:12.550000 8504 Lib\site-packages\torch\_dynamo\convert_frame.py:1233] due to: 
W0312 13:09:12.550000 8504 Lib\site-packages\torch\_dynamo\convert_frame.py:1233] Traceback (most recent call last):
W0312 13:09:12.550000 8504 Lib\site-packages\torch\_dynamo\convert_frame.py:1233]   File "c:\Users\Jason\Documents\GitHub\EZ-Rx-ID\.venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1164, in __call__
W0312 13:09:12.550000 8504 Lib\site-packages\torch\_dynamo\convert_frame.py:1233]     result = self._inner_convert(
W0312 13:09:12.550000 8504 Lib\site-packages\torch\_dynamo\convert_frame.py:1233]              ^^^^^^^^^^^^^^^^^^^^
W0312 13:09:12.550000 8504 Lib\site-packages\torch\

KeyboardInterrupt: 